In [ ]:
from common_all import *
from common_float import *

In [ ]:
def FUNC2(x, y): return func(y, x) / K  # 45 9

In [ ]:
# Assuming that heat source is round and symmetrical
def get_source_effective_radius(cx, cy):
    offset = 0
    while func(cx + offset, cy) > 1e-4:
        offset += 0.1
    return offset

source_eff_ofsset = get_source_effective_radius(45, 9)
print(f'Effective offset: {source_eff_ofsset:.2f}')

effbox_x1 = SOURCE_X0 - source_eff_ofsset
effbox_x2 = SOURCE_X0 + source_eff_ofsset
effbox_y1 = SOURCE_Y0 - source_eff_ofsset
effbox_y2 = SOURCE_Y0 + source_eff_ofsset

In [ ]:
# def pt_line_dist(ptx, pty, ax, ay, bx, by, segm_len):
#     v = (by - ay) * ptx - (bx - ax) * pty + bx * ay - by * ax
#     return abs(v) / segm_len

# Passing segm_len improves speed
# TODO: not tested
def has_intersection_segm_circle(cx, cy, r, ax, ay, bx, by, segm_len):
    if m.hypot(abs(cx - ax), abs(cy - ay)) < r:
        return True
    if m.hypot(abs(cx - bx), abs(cy - by)) < r:
        return True

    # Project AC onto AB = AP:
    # AP = scale * AB / norm(AB), where scale = dot(AC, AB) / norm(AB)
    # TODO: coordinates differences have already computed somewhere before?
    scale = ((cx - ax) * (bx - ax) + (cy - ay) * (by - ay)) / segm_len
    if scale < 0 or scale > 1:  # Nearest point outside of segment
        return False
    scale /= segm_len
    projx, projy = ax + scale * (bx - ax), ay + scale * (by - ay)
    if m.hypot(abs(projx - cx), abs(projy - cy)) < r:
        return True

    return False

In [ ]:
# TODO: not tested
def has_intersection_segm_rect(sx1, sy1, sx2, sy2, ax, ay, bx, by):
    if sx1 <= ax <= sx2 and sy1 <= ay <= sy2:
        return True
    if sx1 <= bx <= sx2 and sy1 <= by <= sy2:
        return True

    def intersect1(a, b, c, d):
        if a > b: a, b = b, a
        if c > d: c, d = d, c
        return max(a, c) <= min(b, d)

    if not intersect1(sx1, sx2, ax, bx) or not intersect1(sy1, sy2, ay, by):
        return False

    def area3(ax, ay, bx, by, cx, cy):
        return (bx - ax) * (cy - ay) - (by - ay) * (cx - ax)

    def instersect_segms(ax, ay, bx, by, cx, cy, dx, dy):
        s1 = area3(ax, ay, bx, by, cx, cy) <= 0
        s2 = area3(ax, ay, bx, by, dx, dy) <= 0
        if s1 == s2: return False
        s1 = area3(cx, cy, dx, dy, ax, ay) <= 0
        s2 = area3(cx, cy, dx, dy, bx, by) <= 0
        return s1 != s2

    if instersect_segms(ax, ay, bx, by, sx1, sy1, sx2, sy2):
        return True
    return instersect_segms(ax, ay, bx, by, sx1, sy2, sx2, sy1)

In [ ]:
def has_effective_intersection(ax, ay, bx, by):
    return has_intersection_segm_rect(
        effbox_x1, effbox_y1, effbox_x2, effbox_y2,
        ax, ay, bx, by)

In [ ]:
# Simpson's integration

INTEGR_STEP = 1e-2

_INTEGR_STEP_2 = INTEGR_STEP / 2.0

def integrate_segm(x1: int, y1: int, x2: int, y2: int):
    dx, dy = abs(x2 - x1), abs(y2 - y1)
    segm_len = m.hypot(dx, dy)
    halfx = _INTEGR_STEP_2 * dx
    halfy = _INTEGR_STEP_2 * dy

    s = 0
    ptx, pty = x1, y1
    lastval = FUNC2(ptx, pty)

    for i in range(int(segm_len / INTEGR_STEP)):
        s += lastval
        s += 4 * FUNC2(ptx + halfx, pty + halfy)

        ptx += INTEGR_STEP
        pty += INTEGR_STEP 
        lastval = FUNC2(ptx, pty)
        s += lastval

    res = s * INTEGR_STEP / 6.0
    return res

In [ ]:
def prob_trace(y: float, x: float, eps: float, fixed_step):
    _2_PI = 2 * m.pi

    hops = 0
    res = 0

    dist, btemp = dist_to_border(y, x)
    while dist > eps and not is_outside(y, x):
        angle = _2_PI * rnd.random()
        # if fixed_step is None:
        #     nx = x + dist * m.cos(angle)
        #     ny = y + dist * m.sin(angle)
        # else:
        nx = x + fixed_step * m.cos(angle)
        ny = y + fixed_step * m.sin(angle)

        if has_effective_intersection(x, y, nx, ny):
            res += integrate_segm(x, y, nx, ny) * 0.25
        else:
            res += PASSIVE_HEATING

        x, y = nx, ny

        dist, btemp = dist_to_border(y, x)
        hops += 1

    res += btemp
    return res

def prob_point(j, i, traces_cnt: int, eps: float, fixed_step):
    res = 0
    for _ in range(traces_cnt):
        res += prob_trace(j, i, eps, fixed_step)
    res /= traces_cnt
    return res

In [ ]:
TARGET = (45, 9)
TRACES_COUNT = 100
EPS = 1e-2

FIXED_STEP = 1

res = prob_point(*TARGET, TRACES_COUNT, EPS, FIXED_STEP)
print(f'Res: {res:.3f}')
orig = 330.6864168849065
err = (abs(res - orig) / orig) * 100
print(f'Err: {err:.3f}%')